In [1]:
import altair as alt
import pandas as pd
import numpy as np
#get the data
from vega_datasets import data

[source](https://medium.com/analytics-vidhya/interactive-data-viz-using-altair-873139771fe2) 

In [ ]:
settle_data = data.seattle_weather()
settle_data.head()
source.head()

In [ ]:
settle_data.weather.unique()

In [ ]:
settle_data.date.unique()

In [ ]:
## Add brush and click

In [6]:
# Defining the color scale for the plot (distribution based on Weather types)
scale = alt.Scale(domain=['sun', 'fog', 'drizzle', 'rain', 'snow'],
                  range=['#e7ba52', '#a7a7a7', '#aec7e8', '#1f77b4', '#9467bd'])
color = alt.Color('weather:N', scale=scale) 
# N : used to specify nominal data

##############################################################################
### Add brush and click
# We create two selections
# - a brush that is active on the top panel
# - a multi-click that is active on the bottom panel
##############################################################################
brush = alt.selection_interval(encodings=['x'])
click = alt.selection_multi(encodings=['color'])

- `T`: Specify Time data
- `Q`: Specify quantitative data
- `O` | Specify ordinal data

## Read Banana

In [4]:
colnames=['block', 'banana_number', 'treatment', 'day', 'humidity', 'temperature', 'black_ratio'] 
b_data = pd.read_csv("banana.csv",names=colnames, header=None)
b_data.head()
b_data.tail()

,block,banana_number,treatment,day,humidity,temperature,black_ratio
155,B,8,control,9,48.0,70.0,0.634690
156,B,1,control,10,46.0,73.0,0.653726
157,B,5,control,10,46.0,73.0,0.653726
158,B,6,control,10,46.0,73.0,0.653726
159,B,8,control,10,46.0,73.0,0.653726


- `T`: Specify Time data
- `Q`: Specify quantitative data
- `O` | Specify ordinal data

In [8]:
b_data.head()

,block,banana_number,treatment,day,humidity,temperature,black_ratio
0,N,1,control,1,37.0,72.0,0.858422
1,N,3,treatment,1,40.0,72.0,0.782818
2,N,7,treatment,1,47.0,72.0,0.788474
3,N,5,control,1,36.0,73.0,0.863295
4,N,2,treatment,1,72.0,40.0,0.790132


In [14]:
brush = alt.selection(type='interval')

points = alt.Chart(b_data).mark_circle().encode(
    alt.X('day:Q', title='Date'),
    alt.Y('black_ratio:Q',
         title='Maximum Daily Temperature (C)',
         scale=alt.Scale(domain=[0.2, 1])
       ),
#     size= 'subject:N', #should be the weight
    color= alt.condition(brush, 'treatment:N', alt.value('lightgray')),
#     size = 'subject:N'
#     color= alt.condition(brush, 'treatment:N', alt.value('lightgray'),legend=alt.Legend(orient="right"))
).add_selection(
    brush
).transform_filter(
    click
)

temp = alt.Chart(b_data).mark_circle().encode(
    alt.X('humidity:Q', title='Date',
          scale=alt.Scale(domain=[0, 90])),
    alt.Y('temperature:Q',
         title='Maximum Daily Temperature (C)',
         scale=alt.Scale(domain=[0, 90])
       ),
#     size= 'subject:N', #should be the weight
    color= alt.condition(brush, 'treatment:N', alt.value('lightgray'),legend=alt.Legend(orient="right"))
).transform_filter(
    brush
).add_selection(
    click
)


points | temp

alt.HConcatChart(...)

In [ ]:
np.random.seed(0)

n_objects = 20
n_times = 50

# Create one (x, y) pair of metadata per object
locations = pd.DataFrame({
    'id': range(n_objects),
    'x': np.random.randn(n_objects),
    'y': np.random.randn(n_objects)
})

locations.head()

In [ ]:
# Create a 50-element time-series for each object
timeseries = pd.DataFrame(np.random.randn(n_times, n_objects).cumsum(0),
                          columns=locations['id'],
                          index=pd.RangeIndex(0, n_times, name='time'))

timeseries.head()

In [ ]:
# Melt the wide-form timeseries into a long-form view
timeseries = timeseries.reset_index().melt('time')
timeseries.head()

In [ ]:
# Merge the (x, y) metadata into the long-form view
timeseries['id'] = timeseries['id'].astype(int)  # make merge not complain
data = pd.merge(timeseries, locations, on='id')

In [ ]:
# Data is prepared, now make a chart

#selector = alt.selection_single(empty='all', fields=['id'])
selector = alt.selection_single(fields=['id'])

base = alt.Chart(data).properties(
    width=250,
    height=250
).add_selection(selector)



points = base.mark_point(filled=True, size=200).encode(
    x='mean(x)',
    y='mean(y)',
    color=alt.condition(selector, 'id:O', alt.value('lightgray'), legend=None),
)

timeseries = base.mark_line().encode(
    x='time',
    y=alt.Y('value', scale=alt.Scale(domain=(-15, 15))),
    color=alt.Color('id:O', legend=None)
).transform_filter(
    selector
)

points | timeseries